In [7]:
import os
from typing import TypedDict, List, Literal
from langgraph.graph import StateGraph, END
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_ollama import ChatOllama, OllamaEmbeddings
from langchain_chroma import Chroma

# Import M3's Persona
from prompts import SOMY_SYSTEM_PROMPT, CRISIS_RESPONSE

# --- CONFIGURATION ---
DB_PATH = "chroma_db" # This folder must exist (From M2)
LLM_MODEL = "llama3.1"
EMBED_MODEL = "nomic-embed-text"

# --- 1. SETUP MODELS ---
print("🔌 Connecting to Brain (Ollama)...")
try:
    llm = ChatOllama(model=LLM_MODEL, temperature=0.7)
except Exception as e:
    print(f"❌ Error connecting to Ollama: {e}")
    llm = None

print("🔌 Connecting to Memory (ChromaDB)...")
# We connect to the DB created by M2. If it doesn't exist, we skip RAG.
retriever = None
if os.path.exists(DB_PATH):
    try:
        embeddings = OllamaEmbeddings(model=EMBED_MODEL)
        vector_store = Chroma(persist_directory=DB_PATH, embedding_function=embeddings)
        retriever = vector_store.as_retriever(search_kwargs={"k": 3}) # Get top 3 pages
        print("   ✅ Database connected.")
    except Exception as e:
        print(f"   ⚠ Database error: {e}")
else:
    print("   ⚠ WARNING: 'chroma_db' folder not found. Running in 'Amnesia Mode' (No RAG).")

# --- 2. DEFINE STATE ---
# This dictionary holds the data as it moves through the graph nodes
class AgentState(TypedDict):
    messages: List[object] # Chat history
    context: str           # Retrieved book info
    is_safe: bool          # Safety flag

# --- 3. DEFINE NODES (The Workers) ---

def safety_check(state: AgentState):
    """Worker 1: The Guardrail (M4 Logic)"""
    # Get the last message from the user
    last_msg = state["messages"][-1].content.lower()
    danger_words = ["die", "kill", "suicide", "end it", "hurt myself"]
    
    # Check if safe
    is_unsafe = any(word in last_msg for word in danger_words)
    
    if is_unsafe:
        print("🚨 GUARDRAIL HIT: Unsafe content detected.")
        
    return {"is_safe": not is_unsafe}

def retrieve_knowledge(state: AgentState):
    """Worker 2: The Librarian (M2 Logic)"""
    if not retriever:
        return {"context": "No medical manuals available."}
    
    print("🔍 Searching Knowledge Base...")
    query = state["messages"][-1].content
    try:
        docs = retriever.invoke(query)
        # Combine the top 3 chunks into one string
        context_text = "\n\n".join([d.page_content for d in docs])
        return {"context": context_text}
    except Exception as e:
        print(f"   ⚠ Retrieval Failed: {e}")
        return {"context": "Error retrieving context."}

def generate_response(state: AgentState):
    """Worker 3: The Psychologist (M3 Logic)"""
    
    # If safety check failed, return crisis message immediately
    if not state["is_safe"]:
        return {"messages": [AIMessage(content=CRISIS_RESPONSE)]}
    
    if not llm:
        return {"messages": [AIMessage(content="System Error: AI Model not connected.")]}

    print("🤖 Generating Response...")
    
    # Inject the retrieved context into the System Prompt
    # If context is empty, it just uses general knowledge
    filled_prompt = SOMY_SYSTEM_PROMPT.format(context=state.get("context", "No manuals."))
    
    # Build the conversation history for the LLM:
    # [System Instruction] + [User Message]
    conversation = [SystemMessage(content=filled_prompt)] + state["messages"]
    
    response = llm.invoke(conversation)
    return {"messages": [response]}

# --- 4. BUILD THE GRAPH ---
workflow = StateGraph(AgentState)

# Add the workers to the graph
workflow.add_node("safety", safety_check)
workflow.add_node("retrieve", retrieve_knowledge)
workflow.add_node("generate", generate_response)

# Define the Flow: Start -> Safety
workflow.set_entry_point("safety")

# Define Conditional Flow (The "Traffic Cop")
def route_safety(state: AgentState) -> Literal["retrieve", "generate"]:
    if state["is_safe"]:
        return "retrieve" # If safe, go get knowledge
    else:
        return "generate" # If unsafe, skip to generation (which will output crisis msg)

workflow.add_conditional_edges("safety", route_safety)

# Normal Flow
workflow.add_edge("retrieve", END)
workflow.add_edge("generate", END)

# Compile the machine
workflow.compile()

🔌 Connecting to Brain (Ollama)...
🔌 Connecting to Memory (ChromaDB)...
   ⚠ WARNING: 'chroma_db' folder not found. Running in 'Amnesia Mode' (No RAG).


ValueError: Failed to reach https://mermaid.ink API while trying to render your graph. Status code: 204.

To resolve this issue:
1. Check your internet connection and try again
2. Try with higher retry settings: `draw_mermaid_png(..., max_retries=5, retry_delay=2.0)`
3. Use the Pyppeteer rendering method which will render your graph locally in a browser: `draw_mermaid_png(..., draw_method=MermaidDrawMethod.PYPPETEER)`

In [1]:
import os
from typing import TypedDict, List, Literal
from langgraph.graph import StateGraph, END
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_ollama import ChatOllama, OllamaEmbeddings

from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore # <-- Added Pinecone
from ai_router import router_node

# Import M3's Persona
from prompts import SOMY_SYSTEM_PROMPT, CRISIS_RESPONSE

# --- CONFIGURATION ---
PINECONE_API_KEY = "pcsk_2YdHFw_67VFCkBgYApLCgajHUvuQEcgvjp2j6hX9sCetXd8RcLc2xumsEokE4zUpSSag8C" # <-- New Config (M1 needs this from M2)
INDEX_NAME = "somy-ali-brain"
LLM_MODEL = "llama3.1"
EMBED_MODEL = "nomic-embed-text"

# --- 1. SETUP MODELS ---
print("🔌 Connecting to Brain (Ollama)...")
try:
    llm = ChatOllama(model=LLM_MODEL, temperature=0.7)
except Exception as e:
    print(f"❌ Error connecting to Ollama: {e}")
    llm = None

print("🔌 Connecting to Memory (pinecone)...")
# We connect to the DB created by M2. If it doesn't exist, we skip RAG.
retriever = None
try:
    # Initialize Pinecone Client
    pc = Pinecone(api_key=PINECONE_API_KEY)
    
    # Initialize Embeddings
    embeddings = OllamaEmbeddings(model=EMBED_MODEL)
    
    # Connect to the Index
    vector_store = PineconeVectorStore(
        index_name=INDEX_NAME,
        embedding=embeddings,
        pinecone_api_key=PINECONE_API_KEY
    )
    
    retriever = vector_store.as_retriever(search_kwargs={"k": 3})
    print("   ✅ Connected to Pinecone Index.")

except Exception as e:
    print(f"   ⚠️ Pinecone Error: {e}")
    print("   Running in 'Amnesia Mode' (No RAG).")

# --- 2. DEFINE STATE ---
# This dictionary holds the data as it moves through the graph nodes
class AgentState(TypedDict):
    messages: List[object] # Chat history
    context: str           # Retrieved book info
    is_safe: bool          # Safety flag
    filter_category: str  # From Router

# --- 3. DEFINE NODES (The Workers) ---

def safety_check(state: AgentState):
    """Worker 1: The Guardrail (M4 Logic)"""
    # Get the last message from the user
    # We check if messages list is not empty
    if not state["messages"]:
        return {"is_safe": True}
    
    last_msg = state["messages"][-1].content.lower()
    danger_words = [
    # Core suicide terms
    "suicide",
    "suicidal",
    "kill myself",
    "end my life",
    "take my own life",
    "want to die",
    "wish I were dead",
    "death is the only solution",
    "self-harm",
    "self harm",
    "hurt myself",
    "harm myself",
    "cut myself",
    "cutting",
    "bleed to death",
    "cut my veins",
    "smash my head",
    "punish myself",
    "overdose",
    "poison myself",
    "jump off",
    "jump in front of a train",
    "drown myself",
    "suffocate myself",
    "end it all",
    "end the pain",
    "end my suffering",
    "stop existing",
    "disappear forever",
    "vanish from this world",
    "escape life",
    "free from life",
    "life is meaningless",
    "life is not worth living",
    "no reason to live",
    "no future for me",
    "tired of being alive",
    "done with life",
    "giving up on life",
    "I am a burden",
    "everyone would be better without me",
    "no one would miss me",
    "tonight",
    "right now",
    "anymore"
]
    
    # Check if safe
    is_unsafe = any(word in last_msg for word in danger_words)
    
    if is_unsafe:
        print("🚨 GUARDRAIL HIT: Unsafe content detected.")
        
    return {"is_safe": not is_unsafe}

def retrieve_knowledge(state: AgentState):
    """Worker 2: The Smart Librarian (Updated for Week 3)"""
    if not retriever:
        return {"context": "No medical manuals available."}
    
    print("🔍 Searching Knowledge Base...")
    user_text = state["messages"][-1].content.lower()
    
    # --- METADATA ROUTING LOGIC ---
    # We check the user's text for keywords to decide which "Book" to open.
    # This matches the tags M2 created in 'ingest_with_tags.py'.
    
    target_category = state.get("filter_category", "general")
    filter_dict = None # Default: Search everything
    
    # if "child" in user_text or "teen" in user_text or "son" in user_text or "daughter" in user_text:
    #     print("   🏷️ Filter Applied: Audience='child'")
    #     filter_dict = {"audience": "child"}
        
    # elif "panic" in user_text or "anxiety" in user_text or "worry" in user_text:
    #     print("   🏷️ Filter Applied: Category='anxiety'")
    #     filter_dict = {"category": "anxiety"}
        
    # elif "sad" in user_text or "depress" in user_text or "hopeless" in user_text:
    #     print("   🏷️ Filter Applied: Category='depression'")
    #     filter_dict = {"category": "depression"}

    # elif "trauma" in user_text or "ptsd" in user_text or "abuse" in user_text:
    #     print("   🏷️ Filter Applied: Category='trauma'")
    #     filter_dict = {"category": "trauma"}
    # elif "anger" in user_text:
    #     filter_dict = {"category": "anger"}
    # elif "grief" in user_text:
    #     filter_dict = {"category": "grief"}
    # elif "sleep" in user_text:
    #     filter_dict = {"category": "sleep"}
    # elif "addiction" in user_text or "substance" in user_text:
    #     filter_dict = {"category": "addiction"}
#-------Router which replace the above keyword matching logic-------#

    if target_category == "child":
        print("   🏷️ Filter: Audience='child'")
        filter_dict = {"audience": "child"}
    elif target_category != "general":
        print(f"   🏷️ Filter: Category='{target_category}' OR 'general'")
        filter_dict = {
            "$or": [
                {"category": target_category},
                {"category": "general"} 
            ]
        }
    else:
        print("   🏷️ No specific filter (General).")
        
        
    # --- EXECUTE SEARCH ---
    try:
        # We pass the 'filter' to the retriever
        if filter_dict:
            docs = retriever.invoke(user_text, filter=filter_dict)
        else:
            docs = retriever.invoke(user_text) # Search everything if no keyword found
            
        # Combine results
        if not docs:
            return {"context": "No specific manual found for this topic."}
            
        context_text = "\n\n".join([d.page_content for d in docs])
        return {"context": context_text}
        
    except Exception as e:
        print(f"   ⚠️ Retrieval Failed: {e}")
        return {"context": "Error retrieving context."}

# def retrieve_knowledge(state: AgentState):
#     """Worker 2: The Librarian (M2 Logic)"""
#     if not retriever:
#         return {"context": "No medical manuals available."}
    
#     print("🔍 Searching Knowledge Base...")
#     query = state["messages"][-1].content
#     try:
#         docs = retriever.invoke(query)
#         # Combine the top 3 chunks into one string
#         context_text = "\n\n".join([d.page_content for d in docs])
#         return {"context": context_text}
#     except Exception as e:
#         print(f"   ⚠ Retrieval Failed: {e}")
#         return {"context": "Error retrieving context."}

def generate_response(state: AgentState):
    """Worker 3: The Psychologist (M3 Logic)"""
    
    # If safety check failed, return crisis message immediately
    if not state["is_safe"]:
        return {"messages": [AIMessage(content=CRISIS_RESPONSE)]}
    
    if not llm:
        return {"messages": [AIMessage(content="System Error: AI Model not connected.")]}

    print("🤖 Generating Response...")
    
    # Inject the retrieved context into the System Prompt
    # If context is empty, it just uses general knowledge
    filled_prompt = SOMY_SYSTEM_PROMPT.format(context=state.get("context", "No manuals."))
    
    # Build the conversation history for the LLM:
    # We create a new list for the LLM call to ensure structure is correct
    conversation_for_llm = [SystemMessage(content=filled_prompt)] + state["messages"]
    
    try:
        response = llm.invoke(conversation_for_llm)
        return {"messages": [response]}
    except Exception as e:
        print(f"❌ Error invoking LLM: {e}")
        return {"messages": [AIMessage(content="I am having trouble finding the right words. Can you say that again?")]}

# --- 4. BUILD THE GRAPH ---
workflow = StateGraph(AgentState)

# Add the workers to the graph
workflow.add_node("safety", safety_check)
workflow.add_node("router", router_node)
workflow.add_node("retrieve", retrieve_knowledge)
workflow.add_node("generate", generate_response)

# Define the Flow: Start -> Safety
workflow.set_entry_point("safety")

# Define Conditional Flow (The "Traffic Cop")
def route_safety(state: AgentState) -> Literal["router", "generate"]:
    if state["is_safe"]:
        return "router" # If safe, go get knowledge -- gp through router
    else:
        return "generate" # If unsafe, skip to generation (which will output crisis msg)

workflow.add_conditional_edges("safety", route_safety)

# Normal Flow
workflow.add_edge("router", "retrieve")
workflow.add_edge("retrieve", "generate")
workflow.add_edge("generate", END)

# Compile the machine
app_graph = workflow.compile()


c:\Users\Shaziya khan\Desktop\Somyali_backend_project - Copy\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🔌 Connecting to Brain (Ollama)...
🔌 Connecting to Memory (pinecone)...
   ✅ Connected to Pinecone Index.


In [2]:
app_graph

ValueError: Failed to reach https://mermaid.ink API while trying to render your graph. Status code: 204.

To resolve this issue:
1. Check your internet connection and try again
2. Try with higher retry settings: `draw_mermaid_png(..., max_retries=5, retry_delay=2.0)`
3. Use the Pyppeteer rendering method which will render your graph locally in a browser: `draw_mermaid_png(..., draw_method=MermaidDrawMethod.PYPPETEER)`

In [2]:
nam

'Shadman'

In [4]:
import os
from typing import TypedDict, List, Literal
from langgraph.graph import StateGraph, END
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_ollama import ChatOllama, OllamaEmbeddings
from langchain_chroma import Chroma

# Import M3's Persona
from prompts import SOMY_SYSTEM_PROMPT, CRISIS_RESPONSE

# --- CONFIGURATION ---
DB_PATH = "..\chroma_db" # This folder must exist (From M2)
LLM_MODEL = "llama3.1"
EMBED_MODEL = "nomic-embed-text"

# --- 1. SETUP MODELS ---
print("🔌 Connecting to Brain (Ollama)...")
try:
    llm = ChatOllama(model=LLM_MODEL, temperature=0.7)
except Exception as e:
    print(f"❌ Error connecting to Ollama: {e}")
    llm = None

print("🔌 Connecting to Memory (ChromaDB)...")
# We connect to the DB created by M2. If it doesn't exist, we skip RAG.
retriever = None
if os.path.exists(DB_PATH):
    try:
        embeddings = OllamaEmbeddings(model=EMBED_MODEL)
        vector_store = Chroma(persist_directory=DB_PATH, embedding_function=embeddings)
        retriever = vector_store.as_retriever(search_kwargs={"k": 3}) # Get top 3 pages
        print("   ✅ Database connected.")
    except Exception as e:
        print(f"   ⚠ Database error: {e}")
else:
    print("   ⚠ WARNING: 'chroma_db' folder not found. Running in 'Amnesia Mode' (No RAG).")

# --- 2. DEFINE STATE ---
# This dictionary holds the data as it moves through the graph nodes
class AgentState(TypedDict):
    messages: List[object] # Chat history
    context: str           # Retrieved book info
    is_safe: bool          # Safety flag

# --- 3. DEFINE NODES (The Workers) ---

def safety_check(state: AgentState):
    """Worker 1: The Guardrail (M4 Logic)"""
    # Get the last message from the user
    # We check if messages list is not empty
    if not state["messages"]:
        return {"is_safe": True}
    
    last_msg = state["messages"][-1].content.lower()
    danger_words = [
    # Core suicide terms
    "suicide",
    "suicidal",
    "kill myself",
    "end my life",
    "take my own life",
    "want to die",
    "wish I were dead",
    "death is the only solution",
    "self-harm",
    "self harm",
    "hurt myself",
    "harm myself",
    "cut myself",
    "cutting",
    "bleed to death",
    "cut my veins",
    "smash my head",
    "punish myself",
    "overdose",
    "poison myself",
    "jump off",
    "jump in front of a train",
    "drown myself",
    "suffocate myself",
    "end it all",
    "end the pain",
    "end my suffering",
    "stop existing",
    "disappear forever",
    "vanish from this world",
    "escape life",
    "free from life",
    "life is meaningless",
    "life is not worth living",
    "no reason to live",
    "no future for me",
    "tired of being alive",
    "done with life",
    "giving up on life",
    "I am a burden",
    "everyone would be better without me",
    "no one would miss me",
    "tonight",
    "right now",
    "anymore"
]
    
    # Check if safe
    is_unsafe = any(word in last_msg for word in danger_words)
    
    if is_unsafe:
        print("🚨 GUARDRAIL HIT: Unsafe content detected.")
        
    return {"is_safe": not is_unsafe}

def retrieve_knowledge(state: AgentState):
    """Worker 2: The Smart Librarian (Updated for Week 3)"""
    if not retriever:
        return {"context": "No medical manuals available."}
    
    print("🔍 Searching Knowledge Base...")
    user_text = state["messages"][-1].content.lower()
    
    # --- METADATA ROUTING LOGIC ---
    # We check the user's text for keywords to decide which "Book" to open.
    # This matches the tags M2 created in 'ingest_with_tags.py'.
    
    filter_dict = None # Default: Search everything
    
    if "child" in user_text or "teen" in user_text or "son" in user_text or "daughter" in user_text:
        print("   🏷️ Filter Applied: Audience='child'")
        filter_dict = {"audience": "child"}
        
    elif "panic" in user_text or "anxiety" in user_text or "worry" in user_text:
        print("   🏷️ Filter Applied: Category='anxiety'")
        filter_dict = {"category": "anxiety"}
        
    elif "sad" in user_text or "depress" in user_text or "hopeless" in user_text:
        print("   🏷️ Filter Applied: Category='depression'")
        filter_dict = {"category": "depression"}

    elif "trauma" in user_text or "ptsd" in user_text or "abuse" in user_text:
        print("   🏷️ Filter Applied: Category='trauma'")
        filter_dict = {"category": "trauma"}
    elif "anger" in user_text:
        filter_dict = {"category": "anger"}
    elif "grief" in user_text:
        filter_dict = {"category": "grief"}
    elif "sleep" in user_text:
        filter_dict = {"category": "sleep"}
    elif "addiction" in user_text or "substance" in user_text:
        filter_dict = {"category": "addiction"}

    # --- EXECUTE SEARCH ---
    try:
        # We pass the 'filter' to the retriever
        if filter_dict:
            docs = retriever.invoke(user_text, filter=filter_dict)
        else:
            docs = retriever.invoke(user_text) # Search everything if no keyword found
            
        # Combine results
        if not docs:
            return {"context": "No specific manual found for this topic."}
            
        context_text = "\n\n".join([d.page_content for d in docs])
        return {"context": context_text}
        
    except Exception as e:
        print(f"   ⚠️ Retrieval Failed: {e}")
        return {"context": "Error retrieving context."}

# def retrieve_knowledge(state: AgentState):
#     """Worker 2: The Librarian (M2 Logic)"""
#     if not retriever:
#         return {"context": "No medical manuals available."}
    
#     print("🔍 Searching Knowledge Base...")
#     query = state["messages"][-1].content
#     try:
#         docs = retriever.invoke(query)
#         # Combine the top 3 chunks into one string
#         context_text = "\n\n".join([d.page_content for d in docs])
#         return {"context": context_text}
#     except Exception as e:
#         print(f"   ⚠ Retrieval Failed: {e}")
#         return {"context": "Error retrieving context."}

def generate_response(state: AgentState):
    """Worker 3: The Psychologist (M3 Logic)"""
    
    # If safety check failed, return crisis message immediately
    if not state["is_safe"]:
        return {"messages": [AIMessage(content=CRISIS_RESPONSE)]}
    
    if not llm:
        return {"messages": [AIMessage(content="System Error: AI Model not connected.")]}

    print("🤖 Generating Response...")
    
    # Inject the retrieved context into the System Prompt
    # If context is empty, it just uses general knowledge
    filled_prompt = SOMY_SYSTEM_PROMPT.format(context=state.get("context", "No manuals."))
    
    # Build the conversation history for the LLM:
    # [System Instruction] + [User Message]
    conversation = [SystemMessage(content=filled_prompt)] + state["messages"]
    
    response = llm.invoke(conversation)
    return {"messages": [response]}

# --- 4. BUILD THE GRAPH ---
workflow = StateGraph(AgentState)

# Add the workers to the graph
workflow.add_node("safety", safety_check)
workflow.add_node("retrieve", retrieve_knowledge)
workflow.add_node("generate", generate_response)

# Define the Flow: Start -> Safety
workflow.set_entry_point("safety")

# Define Conditional Flow (The "Traffic Cop")
def route_safety(state: AgentState) -> Literal["retrieve", "generate"]:
    if state["is_safe"]:
        return "retrieve" # If safe, go get knowledge
    else:
        return "generate" # If unsafe, skip to generation (which will output crisis msg)

workflow.add_conditional_edges("safety", route_safety)

# Normal Flow
workflow.add_edge("retrieve", "generate")
workflow.add_edge("generate", END)

# Compile the machine
app_graph = workflow.compile()


🔌 Connecting to Brain (Ollama)...
🔌 Connecting to Memory (ChromaDB)...
   ✅ Database connected.


In [6]:
app_graph

ValueError: Failed to reach https://mermaid.ink API while trying to render your graph. Status code: 204.

To resolve this issue:
1. Check your internet connection and try again
2. Try with higher retry settings: `draw_mermaid_png(..., max_retries=5, retry_delay=2.0)`
3. Use the Pyppeteer rendering method which will render your graph locally in a browser: `draw_mermaid_png(..., draw_method=MermaidDrawMethod.PYPPETEER)`